In [1]:
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd

load_dotenv(override=True)

# DATABASE CONNECTION
PG_USER = os.environ.get("PG_USER")
PG_PASS = os.environ.get("PG_PASS")
PG_HOST = os.environ.get("PG_HOST")
PG_DB = os.environ.get("PG_DB")
PG_PORT = os.environ.get("PG_PORT")

conn_string = "postgresql://{0}:{1}@{2}:{3}/{4}".format(
    PG_USER, PG_PASS, PG_HOST, PG_PORT, PG_DB)
engine = create_engine(conn_string)
conn = engine.connect()

In [ ]:
print(conn_string)

In [2]:
county = pd.read_csv('./helper_tables_20200217/county.csv')
county.shape

(100, 2)

In [3]:
county.to_sql('county',
              if_exists='append',
              index=False,
              method='multi',
              chunksize=100, 
              con=conn)

In [4]:
precinct = pd.read_csv('./helper_tables_20200217/precinct.csv')
precinct.shape

(3491, 3)

In [ ]:
precinct.rename(columns = {'name': 'description'}, inplace=True)

In [5]:
precinct[precinct['abbrv'].isna()].shape

(100, 3)

In [ ]:
precinct[precinct['abbrv'].isna()].head()

In [13]:
precinct.fillna({'abbrv': '', 'description': ''}).drop_duplicates(subset=['county_id', 'abbrv']).to_sql('precinct',
                                                                                                        if_exists='append',
                                                                                                        index=False,
                                                                                                        method='multi',
                                                                                                        chunksize=1000,
                                                                                                        con=conn)

In [12]:
precinct[(precinct['county_id'] == 34) & (precinct['abbrv'] == '11.0')]

,county_id,abbrv,description
1028,34,11.0,011
1029,34,11.0,11.0


In [14]:
voter_status = pd.read_csv('./helper_tables_20200217/status.csv')
voter_status.shape

(5, 2)

In [15]:
voter_status.to_sql('voter_status',
                    if_exists='append',
                    index=False,
                    method='multi',
                    chunksize=1000,
                    con=conn)

In [16]:
voter_status_reason = pd.read_csv('./helper_tables_20200217/status_reason.csv')
voter_status_reason.shape

(29, 2)

In [17]:
voter_status_reason.to_sql('voter_status_reason',
                           if_exists='append',
                           index=False,
                           method='multi',
                           chunksize=1000,
                           con=conn)

In [18]:
race = pd.read_csv('./helper_tables_20200217/race.csv')
race.shape

(8, 2)

In [19]:
race

,code,description
0,,NaN
1,A,ASIAN
2,B,BLACK OR AFRICAN AMERICAN
3,I,AMERICAN INDIAN OR ALASKA NATIVE
4,M,TWO OR MORE RACES
5,O,OTHER
6,U,UNDESIGNATED
7,W,WHITE


In [20]:
race.to_sql('race',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000,
            con=conn)

In [21]:
ethnicity = pd.read_csv('./helper_tables_20200217/ethnicity.csv')
ethnicity.shape

(3, 2)

In [22]:
ethnicity.to_sql('ethnicity',
                 if_exists='append',
                 index=False,
                 method='multi',
                 chunksize=1000,
                 con=conn)

In [23]:
party = pd.read_csv('./helper_tables_20200217/party.csv')
party.shape

(6, 2)

In [24]:
party.to_sql('party',
             if_exists='append',
             index=False,
             method='multi',
             chunksize=1000,
             con=conn)

In [25]:
conn.close()
engine.dispose()